<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%202%20-%20Exercise%20-%20Answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
import numpy as np
from keras.layers import Input, Embedding, Dot, Reshape, Add
from keras.models import Model
from keras.optimizers import Adam
import keras.backend as K
import nltk
from nltk.corpus import brown

In [46]:
nltk.download('brown')
data = brown.sents(categories=brown.categories())
len(data)

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


57340

In [0]:
sentences=[]
stopwords_ = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
punctuation = ['!','"','#','$','%','&',"'",'(',')','*','+',',','-','.','/',':',';','<','=','>','?','@','[','\\',']','^','_','`','{','|','}','~','``',"''",'--']
for sentence in data:
    for word in stopwords_:
        token=" "+word+" "
        sentence=[item.replace(token," ") for item in sentence]
    sentences.append(sentence)


In [48]:
for i in range(len(sentences)):
    sentences[i]=[item.lower() for item in sentences[i]]
    sentences[i]=[item*(len(item)>2) for item in sentences[i]]
    for pun in punctuation:
        sentences[i]=[item.replace(pun,"") for item in sentences[i]]
        sentences[i]=[item for item in sentences[i] if item]
print(sentences[100])

['daniel', 'personally', 'led', 'the', 'fight', 'for', 'the', 'measure', 'which', 'had', 'watered', 'down', 'considerably', 'since', 'its', 'rejection', 'two', 'previous', 'legislatures', 'public', 'hearing', 'before', 'the', 'house', 'committee', 'revenue', 'and', 'taxation']


In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
vocab_size=10000
vector_dim=100
maxlen=20
windowSize=3

In [0]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

In [0]:
sequences = tokenizer.texts_to_sequences(sentences)
sequences_padded = pad_sequences(sequences, padding='post', maxlen=maxlen)


In [53]:
print(sentences[4000])
print(sequences[4000])
print(sequences_padded[4000])
print(len(sentences[4000]))
print(len(sequences[4000]))
print(len(sequences_padded[4000]))

['ruths', 'day', 'and', 'until', 'this', 'year', 'the', 'schedule', 'was', '154', 'games']
[9177, 108, 3, 161, 9, 117, 2, 2968, 5, 1, 1826]
[9177  108    3  161    9  117    2 2968    5    1 1826    0    0    0
    0    0    0    0    0    0]
11
11
20


In [0]:
from collections import defaultdict
co_matrix = defaultdict(lambda: defaultdict(int))

In [0]:
for sentence in sequences_padded:
    sentence_size=len(sentence)
    for i in range(sentence_size):
        for distance in range(1,windowSize+1):
            if i+distance<sentence_size:
                if sentence[i]>sentence[i+distance]:
                    first=sentence[i+distance]
                    second=sentence[i]
                else:
                    second=sentence[i+distance]
                    first=sentence[i]
                co_matrix[first][second]+=1.0/distance

In [0]:
first,second,freqs=[],[],[]
for first_id in co_matrix.keys():
    for second_id in co_matrix[first_id].keys():
        freq=co_matrix[first_id][second_id]
        first.append(first_id)
        second.append(second_id)
        freqs.append(freq)
        first.append(second_id)
        second.append(first_id)
        freqs.append(freq)
first=np.array(first)
second=np.array(second)
freqs=np.array(freqs)

In [57]:
print(np.shape(first))
print(np.shape(second))
print(np.shape(freqs))

(1306226,)
(1306226,)
(1306226,)


In [0]:
#time for modelling yay!
#shamelessly copied loss haha
X_MAX = 100
a = 3.0 / 4.0
def customLoss(y_true,y_pred):
    return K.sum(K.pow(K.clip(y_true / X_MAX, 0.0, 1.0), a) * K.square(y_pred - K.log(y_true)), axis=-1)

In [0]:
def modelMake():

    input_target = Input((1,))
    input_context = Input((1,))

    central_embedding = Embedding(vocab_size, vector_dim, input_length=1)
    central_bias = Embedding(vocab_size, 1, input_length=1)

    context_embedding = Embedding(vocab_size, vector_dim, input_length=1)
    context_bias = Embedding(vocab_size, 1, input_length=1)

    vector_target = central_embedding(input_target)
    vector_context = context_embedding(input_context)

    bias_target = central_bias(input_target)
    bias_context = context_bias(input_context)

    dot_product = Dot(axes=-1)([vector_target, vector_context])
    dot_product = Reshape((1, ))(dot_product)
    bias_target = Reshape((1,))(bias_target)
    bias_context = Reshape((1,))(bias_context)

    prediction = Add()([dot_product, bias_target, bias_context])

    model = Model(inputs=[input_target, input_context], outputs=prediction)
    model.compile(loss=customLoss, optimizer=Adam())
    model.summary()
    return Model

In [71]:
model=modelMake()
# model.fit([first, second], freqs, epochs=20, batch_size=1024)
vectors=(model.layers[2].get_weights()+model.layers[3].get_weights())*0.5

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_28 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_53 (Embedding)        (None, 1, 100)       1000000     input_27[0][0]                   
__________________________________________________________________________________________________
embedding_55 (Embedding)        (None, 1, 100)       1000000     input_28[0][0]                   
____________________________________________________________________________________________

TypeError: ignored

In [18]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv \
    -O /tmp/bbc-text.csv
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_portion = .8

--2020-05-22 15:15:27--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 2607:f8b0:400e:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [application/octet-stream]
Saving to: ‘/tmp/bbc-text.csv’

/tmp/bbc-text.csv   100%[===================>]   4.82M  --.-KB/s    in 0.07s   

2020-05-22 15:15:27 (73.6 MB/s) - ‘/tmp/bbc-text.csv’ saved [5057493/5057493]



In [29]:
sentences = []
labels = []
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
print(len(stopwords))
# Expected Output
# 153

153


In [30]:
with open("/tmp/bbc-text.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[0])
        sentence = row[1]
        for word in stopwords:
            token = " " + word + " "
            sentence = sentence.replace(token, " ")
        sentences.append(sentence)

print(len(labels))
print(len(sentences))
print(sentences[0])

2225
2225
tv future hands viewers home theatre systems  plasma high-definition tvs  digital video recorders moving living room  way people watch tv will radically different five years  time.  according expert panel gathered annual consumer electronics show las vegas discuss new technologies will impact one favourite pastimes. us leading trend  programmes content will delivered viewers via home networks  cable  satellite  telecoms companies  broadband service providers front rooms portable devices.  one talked-about technologies ces digital personal video recorders (dvr pvr). set-top boxes  like us s tivo uk s sky+ system  allow people record  store  play  pause forward wind tv programmes want.  essentially  technology allows much personalised tv. also built-in high-definition tv sets  big business japan us  slower take off europe lack high-definition programming. not can people forward wind adverts  can also forget abiding network channel schedules  putting together a-la-carte entertai

In [31]:
train_size = int(len(sentences) * training_portion)

train_sentences = sentences[:train_size]
train_labels = labels[:train_size]

validation_sentences = sentences[train_size:]
validation_labels = labels[train_size:]

print(train_size)
print(len(train_sentences))
print(len(train_labels))
print(len(validation_sentences))
print(len(validation_labels))

1780
1780
1780
445
445


In [32]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

print(len(train_sequences[0]))
print(len(train_padded[0]))

print(len(train_sequences[1]))
print(len(train_padded[1]))

print(len(train_sequences[10]))
print(len(train_padded[10]))

449
120
200
120
192
120


In [33]:
validation_sequences = tokenizer.texts_to_sequences(validation_sentences)
validation_padded = pad_sequences(validation_sequences, padding=padding_type, maxlen=max_length)

print(len(validation_sequences))
print(validation_padded.shape)

445
(445, 120)


In [34]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))

print(training_label_seq[0])
print(training_label_seq[1])
print(training_label_seq[2])
print(training_label_seq.shape)

print(validation_label_seq[0])
print(validation_label_seq[1])
print(validation_label_seq[2])
print(validation_label_seq.shape)

[4]
[2]
[1]
(1780, 1)
[5]
[4]
[3]
(445, 1)


In [35]:
Model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, vector_dim, input_length=max_length,traina),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])
Model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
Model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 100)          1000000   
_________________________________________________________________
global_average_pooling1d_2 ( (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 24)                2424      
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 150       
Total params: 1,002,574
Trainable params: 1,002,574
Non-trainable params: 0
_________________________________________________________________


In [27]:
num_epochs = 30
history = Model.fit(train_padded, training_label_seq, epochs=num_epochs, validation_data=(validation_padded, validation_label_seq), verbose=2)

ValueError: ignored

In [0]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "acc")
plot_graphs(history, "loss")

In [0]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_sentence(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])


In [0]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

# Expected output
# (1000, 16)